In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input/playground-series-s5e6'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)

print("Libraries imported successfully!")
print(f"XGBoost version: {xgb.__version__}")


In [ ]:
# MAP@3 implementation
def apk(actual, predicted, k=3):
    if len(predicted) > k:
        predicted = predicted[:k]
    
    score = 0.0
    num_hits = 0.0
    
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    
    if not actual:
        return 0.0
    
    return score / min(len(actual), k)

def mapk(actual, predicted, k=3):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

def map3_score_from_proba(y_true, y_pred_proba):
    top3_indices = np.argsort(y_pred_proba, axis=1)[:, ::-1][:, :3]
    
    map3_scores = []
    for i, true_label in enumerate(y_true):
        predicted_labels = top3_indices[i]
        map3_scores.append(apk([true_label], predicted_labels, k=3))
    
    return np.mean(map3_scores)

print("MAP@3 evaluation functions defined!")


In [ ]:
# Load data
train_df = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')
sample_submission = pd.read_csv('/kaggle/input/playground-series-s5e6/sample_submission.csv')

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"Sample submission shape: {sample_submission.shape}")


In [ ]:
def create_ultra_competitive_features(df):
    df = df.copy()
    
    # Fix column name typo
    if 'Temparature' in df.columns:
        df = df.rename(columns={'Temparature': 'Temperature'})
    
    # 1. CATEGORICAL VERSIONS (+0.006)
    numerical_cols = ['Temperature', 'Humidity', 'Moisture', 'Nitrogen', 'Phosphorous', 'Potassium']
    for col in numerical_cols:
        if col in df.columns:
            df[f'{col}_cat'] = pd.cut(df[col], bins=20, labels=False, duplicates='drop')
        
    # 2. CONSTANT FEATURE (+0.005)
    df['const'] = 1
    
    # 3. ENVIRONMENTAL FEATURES
    env_cols = [col for col in ['Temperature', 'Humidity', 'Moisture'] if col in df.columns]
    if len(env_cols) >= 2:
        df['env_max'] = df[env_cols].max(axis=1)
        df['env_min'] = df[env_cols].min(axis=1)
        df['env_range'] = df['env_max'] - df['env_min']
        df['climate_comfort'] = df[env_cols].mean(axis=1)
    
    if 'Temperature' in df.columns and 'Humidity' in df.columns:
        df['temp_humidity_index'] = df['Temperature'] * df['Humidity'] / 100
    
    # 4. NPK RATIOS (CRITICAL)
    epsilon = 1e-8
    npk_cols = ['Nitrogen', 'Phosphorous', 'Potassium']
    
    if all(col in df.columns for col in npk_cols):
        df['N_P_ratio'] = df['Nitrogen'] / (df['Phosphorous'] + epsilon)
        df['N_K_ratio'] = df['Nitrogen'] / (df['Potassium'] + epsilon)
        df['P_K_ratio'] = df['Phosphorous'] / (df['Potassium'] + epsilon)
        df['Total_NPK'] = df['Nitrogen'] + df['Phosphorous'] + df['Potassium']
        df['NPK_balance'] = df[npk_cols].std(axis=1)
        
        # Clip extreme ratios
        ratio_cols = ['N_P_ratio', 'N_K_ratio', 'P_K_ratio']
        for col in ratio_cols:
            df[col] = np.clip(df[col], 0, 100)
            
        # NPK dominance features
        df['N_dominance'] = df['Nitrogen'] / (df['Total_NPK'] + epsilon)
        df['P_dominance'] = df['Phosphorous'] / (df['Total_NPK'] + epsilon)  
        df['K_dominance'] = df['Potassium'] / (df['Total_NPK'] + epsilon)
        
    # 5. TEMPERATURE SUITABILITY
    if 'Temperature' in df.columns and 'Crop Type' in df.columns:
        crop_temp_map = {
            'Sugarcane': (26, 35), 'Maize': (25, 32), 'Wheat': (20, 30),
            'Paddy': (25, 35), 'Cotton': (25, 35), 'Tobacco': (20, 30),
            'Barley': (15, 25), 'Millets': (25, 35), 'Pulses': (20, 30),
            'Oil seeds': (20, 30), 'Ground Nuts': (25, 32)
        }
        
        def temp_suitable(row):
            temp_range = crop_temp_map.get(row['Crop Type'], (25, 32))
            return 1 if temp_range[0] <= row['Temperature'] <= temp_range[1] else 0
            
        df['temp_suitability'] = df.apply(temp_suitable, axis=1)
    
    # 6. CROP-SOIL INTERACTIONS
    if 'Crop Type' in df.columns and 'Soil Type' in df.columns:
        df['Crop_Soil_combo'] = df['Crop Type'].astype(str) + '_' + df['Soil Type'].astype(str)
    
    # 7. STRESS INDICATORS
    if 'Temperature' in df.columns:
        df['temp_stress'] = np.where((df['Temperature'] < 20) | (df['Temperature'] > 35), 1, 0)
    if 'Humidity' in df.columns:
        df['humidity_stress'] = np.where((df['Humidity'] < 40) | (df['Humidity'] > 80), 1, 0)
    if 'Moisture' in df.columns:
        df['moisture_stress'] = np.where((df['Moisture'] < 30) | (df['Moisture'] > 70), 1, 0)
    
    # 8. NUTRIENT EFFICIENCY
    if all(col in df.columns for col in npk_cols):
        df['nutrient_efficiency'] = df['Total_NPK'] / (df['Temperature'] + df['Humidity'] + df['Moisture'] + epsilon)
    
    return df

print("Ultra-competitive feature engineering function defined!")


In [ ]:
# Apply feature engineering
print("Applying feature engineering...")
X_train = train_df.drop(['id', 'Fertilizer Name'], axis=1)
y_train = train_df['Fertilizer Name']

X_train_engineered = create_ultra_competitive_features(X_train)

# Label encode target
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

print(f"Original features: {X_train.shape[1]}")
print(f"Engineered features: {X_train_engineered.shape[1]}")
print(f"Target classes: {len(label_encoder.classes_)}")

# Handle categorical columns
categorical_cols = ['Soil Type', 'Crop Type', 'Crop_Soil_combo']
for col in categorical_cols:
    if col in X_train_engineered.columns:
        X_train_engineered[col] = X_train_engineered[col].astype('category')

print("Feature engineering completed!")


In [ ]:
# 4x Data expansion
print("Expanding training data by factor of 4...")
X_train_expanded = pd.concat([X_train_engineered] * 4, ignore_index=True)
y_train_expanded = np.tile(y_train_encoded, 4)

print(f"Original size: {len(y_train_encoded)} -> Expanded size: {len(y_train_expanded)}")
print("Data expansion completed!")


In [ ]:
# Fixed parameters from overfitted trial (but with proper validation)
FIXED_PARAMS = {
    'objective': 'multi:softprob',
    'num_class': 7,
    'eval_metric': 'mlogloss',
    'tree_method': 'gpu_hist',
    'enable_categorical': True,
    'random_state': 42,
    'verbosity': 0,
    'gpu_id': 0,
    
    # Parameters from overfitted trial
    'max_depth': 7,
    'learning_rate': 0.2536999076681772,
    'n_estimators': 2330,
    'subsample': 0.8394633936788146,
    'colsample_bytree': 0.6624074561769746,
    'colsample_bylevel': 0.662397808134481,
    'reg_alpha': 0.014936568554617643,
    'reg_lambda': 3.9676050770529883,
    'min_child_weight': 7,
    'gamma': 3.540362888980227
}

print("Fixed parameters loaded from overfitted trial:")
for param, value in FIXED_PARAMS.items():
    if param not in ['objective', 'num_class', 'eval_metric', 'tree_method', 'enable_categorical', 'random_state', 'verbosity', 'gpu_id']:
        print(f"  {param}: {value}")


In [ ]:
# Proper cross-validation with fixed parameters
print("Performing 5-fold cross-validation with fixed parameters...")

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []
oof_predictions = np.zeros((len(y_train_expanded), 7))

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_expanded, y_train_expanded)):
    print(f"Training fold {fold + 1}/5...")
    
    X_tr, X_val = X_train_expanded.iloc[train_idx], X_train_expanded.iloc[val_idx]
    y_tr, y_val = y_train_expanded[train_idx], y_train_expanded[val_idx]
    
    # Train model with fixed parameters
    model = xgb.XGBClassifier(**FIXED_PARAMS)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=50,
        verbose=False
    )
    
    # Predict and calculate MAP@3
    y_pred_proba = model.predict_proba(X_val)
    oof_predictions[val_idx] = y_pred_proba
    
    map3_score = map3_score_from_proba(y_val, y_pred_proba)
    cv_scores.append(map3_score)
    
    print(f"  Fold {fold + 1} MAP@3: {map3_score:.6f}")

# Overall CV performance
mean_cv_score = np.mean(cv_scores)
std_cv_score = np.std(cv_scores)
oof_score = map3_score_from_proba(y_train_expanded, oof_predictions)

print(f"\n📊 Cross-Validation Results:")
print(f"  Mean CV MAP@3: {mean_cv_score:.6f} ± {std_cv_score:.6f}")
print(f"  OOF MAP@3: {oof_score:.6f}")
print(f"  Individual folds: {[f'{score:.6f}' for score in cv_scores]}")

if mean_cv_score > 0.40:
    print(f"  ⚠️ HIGH SCORE - Likely overfitted!")
elif mean_cv_score > 0.38:
    print(f"  🏆 EXCELLENT - Competitive performance!")
elif mean_cv_score > 0.35:
    print(f"  ✅ GOOD - Solid performance!")
else:
    print(f"  📈 MODERATE - Room for improvement")


In [ ]:
# Train final ensemble (5 models with different seeds)
print("Training final ensemble with 5 different seeds...")

final_models = []
seeds = [42, 123, 456, 789, 999]

for i, seed in enumerate(seeds):
    print(f"Training model {i+1}/5 with seed {seed}...")
    
    # Update random seed
    params = FIXED_PARAMS.copy()
    params['random_state'] = seed
    
    model = xgb.XGBClassifier(**params)
    model.fit(X_train_expanded, y_train_expanded, verbose=False)
    
    final_models.append(model)

print(f"✅ Ensemble of {len(final_models)} models trained!")


In [ ]:
# Prepare test data and make predictions
print("Preparing test data...")

X_test = test_df.drop('id', axis=1, errors='ignore')
X_test_engineered = create_ultra_competitive_features(X_test)

# Handle categorical variables
for col in categorical_cols:
    if col in X_test_engineered.columns:
        X_test_engineered[col] = X_test_engineered[col].astype('category')

print(f"Test data shape after engineering: {X_test_engineered.shape}")

# Make ensemble predictions
print("Making ensemble predictions...")

all_test_predictions = []
for i, model in enumerate(final_models):
    print(f"Predicting with model {i+1}/{len(final_models)}...")
    pred = model.predict_proba(X_test_engineered)
    all_test_predictions.append(pred)

# Ensemble average
test_probabilities = np.mean(all_test_predictions, axis=0)

# Get top 3 predictions
top3_predictions = np.argsort(test_probabilities, axis=1)[:, ::-1][:, :3]

# Convert back to fertilizer names
top3_fertilizer_names = []
for i in range(len(top3_predictions)):
    fertilizer_names = [label_encoder.inverse_transform([pred])[0] for pred in top3_predictions[i]]
    top3_fertilizer_names.append(fertilizer_names)

print("✅ Predictions completed!")


In [ ]:
# Create submission
print("Creating submission file...")

submission = pd.DataFrame()
submission['id'] = test_df['id']

# Correct format: space-separated top 3 predictions in single column
submission['Fertilizer Name'] = [' '.join(pred) for pred in top3_fertilizer_names]

print("\n📋 First 10 predictions:")
print(submission.head(10))

# Save submission
submission_filename = '/kaggle/working/fixed_params_submission.csv'
submission.to_csv(submission_filename, index=False)
print(f"\n💾 Submission saved as: {submission_filename}")

# Also save backup
submission.to_csv('fixed_params_submission.csv', index=False)
print("💾 Backup submission saved to current directory")

# Performance summary
print(f"\n📈 Fixed Parameters Model Performance Summary:")
print(f"  • CV MAP@3: {mean_cv_score:.6f} ± {std_cv_score:.6f}")
print(f"  • OOF MAP@3: {oof_score:.6f}")
print(f"  • Features used: {X_train_engineered.shape[1]}")
print(f"  • Training samples: {len(y_train_expanded):,} (4x expanded)")
print(f"  • Ensemble models: {len(final_models)}")
print(f"  • Test predictions: {len(submission):,}")

print(f"\n🎯 Expected vs Overfitted Performance:")
print(f"  • Overfitted trial score: 0.460 (unrealistic)")
print(f"  • Proper CV score: {mean_cv_score:.6f} (realistic)")
print(f"  • Champion score to beat: 0.383")

if mean_cv_score >= 0.383:
    print(f"  🏆 LIKELY TO BEAT CHAMPION SCORE!")
elif mean_cv_score >= 0.380:
    print(f"  🥈 VERY COMPETITIVE!")
elif mean_cv_score >= 0.375:
    print(f"  🥉 STRONG PERFORMANCE!")
else:
    print(f"  📈 REASONABLE PERFORMANCE")
